# RenewableUK battery storage site data

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pyproj
import re
import os
os.chdir('/Users/simon/Desktop/uk-map/')
%matplotlib inline

In [3]:
wind = pd.read_csv('RenewableUK PI Onshore Wind Projects Operational 2019-04-08.csv')
storage = pd.read_csv('RenewableUK PI Storage Projects Operational 2019-04-08.csv')
store2 = pd.read_csv('RenewableUK PI Projects Under Construction 2019-04-11.csv')


# only contains three projects
# retired_battery = pd.read_csv('RenewableUK PI Storage Projects Ceased Operating 2019-04-08.csv')

In [4]:
# merge in the short list of under construction storage sites

storage = storage.append(store2, ignore_index=True)

# drop rows without data
storage.dropna(subset=['Project_Capacity_MW'], inplace=True)

#storage = storage.reindex()

In [5]:
# add missing storage capacity estimates

# Foyers figure from here:
# https://www.webarchive.org.uk/wayback/archive/20170113164926/http://www.gov.scot/Publications/2010/10/28091356/4

storage.Storage_Capacity_MWh[storage.Project == 'Foyers Pumped Storage'] = 6300

# calculate discharge time for lithium ion batteries

storage['Time'] = storage['Storage_Capacity_MWh'][
    storage.Generator_Type == 'Battery: Lithium-ion'] / storage[
        'Project_Capacity_MW'][storage.Generator_Type ==
                               'Battery: Lithium-ion']

# stratify battery sizes

storage['size'] = np.where(storage.Project_Capacity_MW <1,'tiny',np.nan)
storage['size'] = np.where(((storage.Project_Capacity_MW >=1)),'medium',storage['size'])
storage['size'] = np.where(((storage.Project_Capacity_MW >=10)),'large',storage['size'])

# find average time by size

discharge = storage[['Time','size']].groupby('size').mean().reset_index()
discharge.columns = ['size','avg']

# merge in the average per size bucket

storage = pd.merge(storage,discharge,left_on=['size'],right_on=['size'],how='left')



/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [6]:
# use the average discharge time if there is no figure

storage['Time'] = np.where(storage['Storage_Capacity_MWh'].isnull(),
                           storage['avg'], storage['Time'])

# use this estimated discharge time to calculate MWh

storage['Storage_Capacity_MWh'] = np.where(
    storage['Storage_Capacity_MWh'].isnull(),
    storage['Time'] * storage['Project_Capacity_MW'],
    storage['Storage_Capacity_MWh'])

In [7]:
# convert dates to date format

storage['Commissioned_Date'] = pd.to_datetime(storage['Commissioned_Date'], dayfirst=True)
storage['Construction_Date'] = pd.to_datetime(storage['Construction_Date'], dayfirst=True)


In [8]:
storage['Year Open'] = storage.Commissioned_Date.dt.year
storage['Year Started'] = storage.Construction_Date.dt.year

# use 2019 opening for projects under construction

storage['Year Open'][storage['Year Open'].isnull()] = 2019


/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [9]:
#storage[storage.Commissioned_Date.isnull()]

In [10]:
# select columns we want

storage = storage[[
    'Project', 'Project_Capacity_MW', 'Storage_Capacity_MWh', 'Time',
    'Generator_Type', 'UK_Region', 'Latitude_Decimal',
    'Longitude_Decimal', 'Year Open', 'Year Started','Owner'
]]

# rename the columns to match other datasets

storage.columns = [
    'Site', 'Capacity (MW)', 'Capacity (MWh)', 'Time', 'Type', 'Region',
    'Latitude', 'Longitude', 'Year Open', 'Year Started','Company'
]

In [11]:
# simplify types

storage['Type'][(storage.Site.str.contains('Battery')&(storage.Type.isnull()))] = 'Battery'
storage['Type'][(storage.Type.str.contains('Battery'))] = 'Battery'
storage['Type'][(storage.Type.str.contains('Pumped'))] = 'Pumped Hydro'

storage['Fuel'] = 'Storage'

storage['yearStart'] = storage['Year Open']

# assume batteries remain open in 2030
# may not be accurate
# this paper suggests perhaps 10 year lifetime
# https://www.nrel.gov/docs/fy17osti/67102.pdf

storage['yearEnd'] = 2019

# categorise as low carbon and renewable
# not accurate but best for viewing on the map

storage['Low-Carbon'] = 'Yes'
storage['Renewable'] = 'Yes'
storage['CHP'] = 'No'

# exclude the v small amount of capacity <1MW

storage = storage[storage['Capacity (MW)']>=1]

/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [12]:
# save the file

storage.to_csv('storage.csv',index=False,encoding='utf-8')

# analysis and checks

In [68]:
storage.columns


Index([u'Project', u'Project_Capacity_MW', u'Type', u'Storage_Capacity_MWh',
       u'Developer', u'Owner', u'Generator_Make', u'Generator_Model',
       u'Number_of_Generators', u'Generator_Capacity', u'Generator_Type',
       u'Country', u'County', u'UK_Region', u'Location', u'Sea_area', u'Lpa',
       u'Latitude_Decimal', u'Longitude_Decimal', u'UKNatGridRef',
       u'Hub_Height', u'Tip_Height', u'Rotor_Diameter', u'Summary_Status',
       u'Status', u'Status_Date', u'SupersedeMethod', u'SupersedeByProject',
       u'SupersededMethod', u'SupersededProject', u'Colocated_Grid',
       u'Colocated_Grid_Detail', u'History', u'Submitted_Date',
       u'Submitted_Note', u'Decision_Date', u'Decision_Note',
       u'Appeal_Decision_Date', u'Appeal_Decision_Note', u'Construction_Date',
       u'Construction_Note', u'Commissioned_Date', u'Commissioned_Note',
       u'ResubmittedByProject', u'ResubmissionOfProject', u'Time', u'size',
       u'avg'],
      dtype='object')

In [72]:
storage['Project_Capacity_MW'][storage['Project_Capacity_MW']<1].sum()

3.5509999999999997

In [63]:
storage[[
    'Project_Capacity_MW', 'Storage_Capacity_MWh', 'Time', 'Generator_Model',
    'size', 'avg'
]][storage.Generator_Type == 'Battery: Lithium-ion'].sort_values(
    by='Project_Capacity_MW')

,Project_Capacity_MW,Storage_Capacity_MWh,Time,Generator_Model,size,avg
70,0.100,0.200000,2.000000,A123 Systems Li-ion,tiny,2.131519
61,0.100,0.213152,2.131519,Lithium-ion Battery,tiny,2.131519
27,0.100,0.200000,2.000000,A123 Systems Li-ion,tiny,2.131519
25,0.200,0.426304,2.131519,Saft Intensium,tiny,2.131519
66,0.240,0.511565,2.131519,Li120,tiny,2.131519
59,0.250,0.250000,1.000000,BYD,tiny,2.131519
47,0.256,0.512000,2.000000,Tesla Powerpack,tiny,2.131519
14,0.300,0.640000,2.133333,Lithium-ion Battery,tiny,2.131519
58,0.315,0.500000,1.587302,Tesla Powerpack,tiny,2.131519
63,0.500,2.100000,4.200000,Tesla Powerpack,tiny,2.131519


In [10]:
print str(wind.Project_Capacity_MW[wind.Project_Capacity_MW >=1].sum())
print str(wind.Project_Capacity_MW[wind.Project_Capacity_MW <1].sum())

12496.29
439.935


In [7]:
wind.head()

,Project,Project_Capacity_MW,Type,Storage_Capacity_MWh,Developer,Owner,Generator_Make,Generator_Model,Number_of_Generators,Generator_Capacity,...,Decision_Date,Decision_Note,Appeal_Decision_Date,Appeal_Decision_Note,Construction_Date,Construction_Note,Commissioned_Date,Commissioned_Note,ResubmittedByProject,ResubmissionOfProject
0,"Abbey Road, 209 Turbine 1",0.250,onshore,NaN,Wind NI Ltd,Wind NI Ltd (100%),Bonus,Bonus,1,0.250,...,16/01/2014,Approved (LPA),NaN,NaN,01/10/2016,Under construction [Estimate],21/01/2017,NaN,NaN,NaN
1,"Abbey Road, 209 Turbine 2",0.250,onshore,NaN,Wind NI Ltd,Albion Community Power plc (100%),Vestas,Vestas V39,1,0.250,...,25/09/2014,Approved (LPA),NaN,NaN,01/08/2015,Under construction [Estimate],21/11/2016,NaN,NaN,NaN
2,Aberffrydlan,0.225,onshore,NaN,Landowner (private or company),Landowner (private or company) (100%),NaN,NaN,1,NaN,...,29/09/2015,Approved (LPA),NaN,NaN,14/01/2016,Under construction [Estimate],01/07/2016,NaN,NaN,NaN
3,Abergwaun Community Turbine,0.225,onshore,NaN,Community Owned Project,Community Owned Project (100%),Endurance,Endurance X29,1,0.225,...,08/01/2014,Refused (LPA),01/08/2014,Approved on appeal,29/06/2015,Under construction,08/10/2015,NaN,NaN,NaN
4,ABP Newport,2.350,onshore,NaN,Associated British Ports,Associated British Ports (100%),Enercon,Enercon E82,1,2.350,...,01/11/2012,Approved (LPA),NaN,NaN,01/11/2013,Under construction [Estimate],01/04/2014,[Estimate],NaN,NaN


In [8]:
wind.columns

Index([u'Project', u'Project_Capacity_MW', u'Type', u'Storage_Capacity_MWh',
       u'Developer', u'Owner', u'Generator_Make', u'Generator_Model',
       u'Number_of_Generators', u'Generator_Capacity', u'Generator_Type',
       u'Country', u'County', u'UK_Region', u'Location', u'Sea_area', u'Lpa',
       u'Latitude_Decimal', u'Longitude_Decimal', u'UKNatGridRef',
       u'Hub_Height', u'Tip_Height', u'Rotor_Diameter', u'Summary_Status',
       u'Status', u'Status_Date', u'SupersedeMethod', u'SupersedeByProject',
       u'SupersededMethod', u'SupersededProject', u'Colocated_Grid',
       u'Colocated_Grid_Detail', u'History', u'Submitted_Date',
       u'Submitted_Note', u'Decision_Date', u'Decision_Note',
       u'Appeal_Decision_Date', u'Appeal_Decision_Note', u'Construction_Date',
       u'Construction_Note', u'Commissioned_Date', u'Commissioned_Note',
       u'ResubmittedByProject', u'ResubmissionOfProject'],
      dtype='object')

In [14]:
retired_battery.Generator_Type.unique()

Index([u'Project', u'Project_Capacity_MW', u'Type', u'Storage_Capacity_MWh',
       u'Developer', u'Owner', u'Generator_Make', u'Generator_Model',
       u'Number_of_Generators', u'Generator_Capacity', u'Generator_Type',
       u'Country', u'County', u'UK_Region', u'Location', u'Sea_area', u'Lpa',
       u'Latitude_Decimal', u'Longitude_Decimal', u'UKNatGridRef',
       u'Hub_Height', u'Tip_Height', u'Rotor_Diameter', u'Summary_Status',
       u'Status', u'Status_Date', u'SupersedeMethod', u'SupersedeByProject',
       u'SupersededMethod', u'SupersededProject', u'Colocated_Grid',
       u'Colocated_Grid_Detail', u'History', u'Submitted_Date',
       u'Submitted_Note', u'Decision_Date', u'Decision_Note',
       u'Appeal_Decision_Date', u'Appeal_Decision_Note', u'Construction_Date',
       u'Construction_Note', u'Commissioned_Date', u'Commissioned_Note',
       u'ResubmittedByProject', u'ResubmissionOfProject'],
      dtype='object')

In [16]:
battery.Generator_Type.unique()

array(['Battery: Lithium-ion', nan, 'Pumped hydropower',
       'Battery: Lead-acid', 'Battery: Redox flow', 'Liquefied air',
       'Battery: Lithium-titanite'], dtype=object)

In [24]:
storage.sort_values(by='Project_Capacity_MW',ascending=False)

,Project,Project_Capacity_MW,Type,Storage_Capacity_MWh,Developer,Owner,Generator_Make,Generator_Model,Number_of_Generators,Generator_Capacity,...,Decision_Date,Decision_Note,Appeal_Decision_Date,Appeal_Decision_Note,Construction_Date,Construction_Note,Commissioned_Date,Commissioned_Note,ResubmittedByProject,ResubmissionOfProject
16,Dinorwig Pumped Storage,1728.000,storage,9100.000,NaN,First Hydro Company (100%),GEC,GEC Francis Reversible,6.0,288.000,...,NaN,NaN,NaN,NaN,01/01/1974,Under construction,09/05/1984,NaN,NaN,NaN
15,Cruachan Pumped Storage,440.000,storage,9680.000,NaN,ScottishPower Generation Ltd (100%),Tbc,Reversible Pump Turbine,4.0,110.000,...,NaN,NaN,NaN,NaN,NaN,NaN,15/10/1965,NaN,NaN,NaN
20,Ffestiniog Pumped Storage,360.000,storage,2160.000,NaN,First Hydro Company (100%),Tbc,Vertical Shaft,4.0,90.000,...,NaN,NaN,NaN,NaN,NaN,NaN,10/08/1963,NaN,NaN,NaN
21,Foyers Pumped Storage,305.000,storage,6300.000,NaN,SSE Generation Ltd (100%),Tbc,Vertical Shaft,2.0,150.000,...,NaN,NaN,NaN,NaN,NaN,NaN,31/12/1975,NaN,NaN,NaN
44,Pelham Battery Storage,49.990,storage,24.995,Statera Energy Limited,Statera Energy Limited (100%),Tbc,Lithium-ion Battery,11.0,4.545,...,18/10/2016,Approved (LPA),NaN,NaN,01/07/2017,Under construction [Estimate],31/12/2017,NaN,NaN,NaN
65,West Burton B Battery Storage,49.000,storage,24.500,EDF Renewables,EDF Renewables (100%),Nidec ASI,Nidec,20.0,2.450,...,30/09/2016,Approved (LPA),NaN,NaN,26/07/2017,Under construction,22/06/2018,NaN,NaN,NaN
54,Roosecote Battery Storage,49.000,storage,50.000,Centrica Distributed Generation Ltd,Centrica Distributed Generation Ltd (100%),Tbc,Lithium-ion Battery,1.0,49.000,...,16/08/2016,Approved (LPA),NaN,NaN,01/03/2017,Under construction,19/12/2018,NaN,NaN,NaN
23,Glassenbury Battery Storage,40.000,storage,27.650,Low Carbon Storage Investment Company Ltd,VLC Energy Ltd (100%),LG Chem,LG Chem Battery Module,7.0,5.714,...,21/10/2016,Approved (LPA),NaN,NaN,NaN,NaN,09/01/2018,NaN,NaN,NaN
51,Port of Tyne Battery Storage,35.000,storage,17.500,Renewable Energy Systems Ltd (RES),Foresight Group LLP (100%),Tbc,Lithium-ion Battery,9.0,3.889,...,21/10/2016,Approved (LPA),NaN,NaN,21/06/2017,Under construction,30/06/2018,NaN,NaN,NaN
1,Aylesford Battery Storage,29.000,storage,29.000,Battery Energy Storage Solutions Ltd,Battery Energy Storage Solutions Ltd (100%),Tbc,Lithium-ion Battery,23.0,1.260,...,05/12/2017,Approved (LPA),NaN,NaN,01/01/2018,Under construction [Estimate],28/02/2018,NaN,NaN,NaN


In [19]:
battery[['Project_Capacity_MW','Storage_Capacity_MWh','Generator_Type']].groupby(['Generator_Type']).sum()

,Project_Capacity_MW,Storage_Capacity_MWh
Generator_Type,,
Battery: Lead-acid,1.400,NaN
Battery: Lithium-ion,535.701,337.627
Battery: Lithium-titanite,1.000,0.500
Battery: Redox flow,0.090,1.080
Liquefied air,5.000,15.000
Pumped hydropower,2833.000,20940.000
